In [29]:
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import pandas as pd
from bs4 import BeautifulSoup
import json
from selenium.common.exceptions import NoSuchElementException
from IPython.display import clear_output

url = "http://biblioteca.unemat.br/"

def biblioteca(consulta=True):

    option = Options()
    option.headless = True
    driver = webdriver.Firefox(options=option)

    driver.get(url)
    print("Waiting for connection: ", end="")

    login = str(input("Digite seu login: "))
    password = str(input("Digite sua senha: "))  

    driver.find_element_by_id("personIdUpper").send_keys(login)
    driver.find_element_by_id("passwordUpper").send_keys(password)
    print("Fazendo login...")
    for i in range(10):
        try:
            driver.find_element_by_id("btnLoginUpper").click()
            break
        except NoSuchElementException as e:
            if i == 9:
                print(f"Tentativa {i+1}/10")
            else:
                print(f"Tentativa {i+1}/10, tentando novamente em 1s")
            time.sleep(1)
    print("Mudando para renovação...")
    for i in range(10):
        try:
            driver.find_element_by_id("mlink1649").click()
            break
        except NoSuchElementException as e:
            if i == 9:
                print(f"Tentativa {i+1}/10")
            else:
                print(f"Tentativa {i+1}/10, tentando novamente em 1s")
            time.sleep(1)

    print("Lendo tabelas...")
    for i in range(10):
        try:
            element = driver.find_element_by_id("searchFields")
            html_content = element.get_attribute("outerHTML")
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')
            df_full = pd.read_html(str(table))[0]
            break
        except ValueError as e:
            if i == 9:
                print(f"Tentativa {i+1}/10")
            else:
                print(f"Tentativa {i+1}/10, tentando novamente em 1s")
            time.sleep(1)
    opt = list(df_full.Dados)
    datas = list(df_full["Data prevista da devolução"])
    options = []
    for index, option in enumerate(opt):
        titulo = option.split("Título:")[1].split("Autor:")[0]
        autor = option.split("Autor:")[1].split("Classificação")[0]
        data = datas[0]
        options.append([index, titulo, autor, data])
    clear_output(wait=True)
    print("Empréstimos:")
    if not consulta:
        print("0: Todos\n")
    for option in options:
        print(f"{option[0]+1}: {option[1]} por {option[2]} - Entregar até {option[3]}")
    if consulta:
        return "Nada a fazer"
    print(f"\n{index+2}: Nenhum")
    escolha = int(input("Escolha para renovar: "))
    if escolha == index+2:
        return "Nada a fazer"
    driver.find_elements_by_class_name("mCheckboxGroup")[escolha].click()
    driver.find_elements_by_id("btnFinalize")
    
    driver.quit()
    
    return("Feito")

In [30]:
biblioteca()

Empréstimos:
1:  Processamento de imagens digitais  por  Gonzalez, Rafael C.  - Entregar até 12/03/2020
2:  Comunicações sem fio : principios e práticas  por  Rappaport, Theodore S.  - Entregar até 12/03/2020


'Nada a fazer'